In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D, ELU
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.layers.core import Lambda

from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import cv2

### Generator

In [110]:
import os
import csv

reduced = True

if reduced == True:
    csv_filepath = 'data-udacity/driving_log_reduced.csv'
else:
    csv_filepath = 'data-udacity/driving_log.csv'
samples = []
with open(csv_filepath) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

samples = samples[1:]
        
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [198]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data-udacity-flipped/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            
            # print("X_train: ", X_train)
            # print("y_train: ", y_train)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

### Models

In [182]:
def resize(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 66, 200)

def halve_size(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, imshape[0] // 2, imshape[1] // 2)

def resize_80_160(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 80, 160)

In [199]:
# NVIDIA End to End Learning Pipeline Model

model = Sequential()

# Crop the images
# TODO: I think the cropping coords here are wrong. height should be 105.6
# to preserve aspect ratio.
model.add(Cropping2D(cropping=((50, 20), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

# Normalise data
# TODO: some people use /255.0 - 0.5. Why?
model.add(Lambda(lambda x: x/127.5 - 1.))


# Conv layer 1, 5x5 kernel to 24@ (from 3@)
# TODO: What does the number of filters MEAN?
# Stride of 2x2
model.add(Convolution2D(24, 36, 104,
                        input_shape=(66,200))
         )

model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 2, 5x5 kernel to 36@
model.add(Convolution2D(36, 18, 50))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 3, 5x5 kernel to 48@
model.add(Convolution2D(48, 10, 32))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 4, 3x3 kernel to 64@
model.add(Convolution2D(64, 3, 12))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 5, 3x3 kernel to 64@
model.add(Convolution2D(64, 3, 3))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Flatten
model.add(Flatten())

# Removed: fully connected layer 1, 1164 neurons

# Fc2, 100 neurons
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Fc3, 50 neurons
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Fc4, 10 neurons
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Output
model.add(Dense(1, activation="tanh"))

# Compile model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001)
model.compile(loss='mean_squared_error',
              optimizer=adam,
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_65 (Cropping2D)       (None, 90, 320, 3)    0           cropping2d_input_65[0][0]        
____________________________________________________________________________________________________
lambda_153 (Lambda)              (None, 66, 200, 3)    0           cropping2d_65[0][0]              
____________________________________________________________________________________________________
lambda_154 (Lambda)              (None, 66, 200, 1)    0           lambda_153[0][0]                 
____________________________________________________________________________________________________
lambda_155 (Lambda)              (None, 66, 200, 1)    0           lambda_154[0][0]                 
___________________________________________________________________________________________

In [200]:
# Comma.ai model
# https://github.com/commaai/research/blob/master/train_steering_model.py

model = Sequential()

# Crop 50 pixels from the top of the image and 20 from the bottom
model.add(Cropping2D(cropping=((50, 20), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

model.add(Lambda(lambda x: (x/127.5) - 1.))

model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))

model.add(Flatten())
# model.add(Dropout(.2))
model.add(ELU())

model.add(Dense(512))
# model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(1))

adam = Adam(lr=0.0001)

model.compile(optimizer=adam, loss="mse", metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_66 (Cropping2D)       (None, 90, 320, 3)    0           cropping2d_input_66[0][0]        
____________________________________________________________________________________________________
lambda_156 (Lambda)              (None, 66, 200, 3)    0           cropping2d_66[0][0]              
____________________________________________________________________________________________________
lambda_157 (Lambda)              (None, 66, 200, 1)    0           lambda_156[0][0]                 
____________________________________________________________________________________________________
lambda_158 (Lambda)              (None, 66, 200, 1)    0           lambda_157[0][0]                 
___________________________________________________________________________________________

Data preprocessing reference: [Geoff Breemer](https://carnd-forums.udacity.com/questions/36045049/answers/36047341)

### Train model

In [201]:
# Train model
batch_size = 32
nb_epoch = 20
# data_augmentation = True
 
checkpointer = ModelCheckpoint(filepath="./tmp/comma-weights.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1, save_best_only=False)
    
model.fit_generator(train_generator, 
                    samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), nb_epoch=nb_epoch,
                    callbacks=[checkpointer])

Epoch 1/20
4608/6428 [====================>.........] - ETA: 8s - loss: 0.0134 - acc: 0.5443

KeyboardInterrupt: 

In [143]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
